In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, InceptionV3, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import numpy as np
import shutil

In [ ]:
# Define main directories
base_dir = '/Users/izzymohamed/Downloads/Dataset for Crop Pest and Disease Detection/CCMT Dataset-Augmented'

# Define crop directories
crop_root = base_dir + '/Cashew'

# Define train and test directories
train_set_dir = crop_root + '/train_set'
test_set_dir = crop_root + '/test_set'

In [ ]:
# Display all directories inside the main root
def list_directories(path):
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))

# Call the function with the path you want to explore
# list_directories('base_root)

In [ ]:
# Function to split training data into training and validation sets
def split_train_val(base_train_dir, train_dir, val_dir, val_split=0.2):
    classes = os.listdir(base_train_dir)
    for cls in classes:
        # print(cls)
        # If cls is .DS_Store continue
        if cls == '.DS_Store':
            continue
        
        class_train_dir = os.path.join(base_train_dir, cls)
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
        
        images = os.listdir(class_train_dir)
        train, val = train_test_split(images, test_size=val_split)
        
        for img in train:
            shutil.copy(os.path.join(class_train_dir, img), os.path.join(train_dir, cls, img))
        for img in val:
            shutil.copy(os.path.join(class_train_dir, img), os.path.join(val_dir, cls, img))

In [ ]:
# Function to create and train model
def create_and_train_model(base_model, train_generator, validation_generator, num_classes, epochs=10, fine_tune_epochs=5):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator
    )
    
    for layer in base_model.layers[-50:]:
        layer.trainable = True
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history_fine = model.fit(
        train_generator,
        epochs=fine_tune_epochs,
        validation_data=validation_generator
    )
    
    return model, history, history_fine

In [ ]:
# Loop through each crop directory and train models
crops = ['Cassava', 'Tomato', 'Cashew', 'Maize']
results = {}

In [ ]:
# Loop through each crop directory and train models
for crop in crops:
    print(f'Processing crop: {crop}')
    
    crop_train_dir = os.path.join(base_dir, crop, 'train_set')
    crop_test_dir = os.path.join(base_dir, crop, 'test_set')
    
    train_dir = f'temp_{crop}_train'
    validation_dir = f'temp_{crop}_val'
    
    split_train_val(crop_train_dir, train_dir, validation_dir)
    
    # ImageDataGenerator for data augmentation and preprocessing
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Data generators
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        crop_test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    num_classes = len(train_generator.class_indices)
    
    # Models to compare
    pretrained_models = {
        'MobileNetV2': MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
        'InceptionV3': InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
        'ResNet50': ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    }

    crop_results = {}

    for model_name, base_model in pretrained_models.items():
        print(f'Training model: {model_name} for crop: {crop}')
        model, history, history_fine = create_and_train_model(base_model, train_generator, validation_generator, num_classes)
        loss, accuracy = model.evaluate(test_generator)
        crop_results[model_name] = {
            'model': model,
            'history': history,
            'history_fine': history_fine,
            'accuracy': accuracy
        }

    results[crop] = crop_results

    # Clean up temporary directories
    shutil.rmtree(train_dir)
    shutil.rmtree(validation_dir)

Processing crop: Cassava
Found 22558 images belonging to 7 classes.
Found 10692 images belonging to 7 classes.
Found 7510 images belonging to 5 classes.
Training model: MobileNetV2 for crop: Cassava
Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


705/705 ━━━━━━━━━━━━━━━━━━━━ 615s 852ms/step - accuracy: 0.5789 - loss: 1.1473 - val_accuracy: 0.6721 - val_loss: 0.8560
Epoch 2/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 601s 849ms/step - accuracy: 0.6769 - loss: 0.8258 - val_accuracy: 0.7045 - val_loss: 0.7577
Epoch 3/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 578s 815ms/step - accuracy: 0.6942 - loss: 0.7747 - val_accuracy: 0.7183 - val_loss: 0.7228
Epoch 4/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 604s 853ms/step - accuracy: 0.7035 - loss: 0.7495 - val_accuracy: 0.7415 - val_loss: 0.6507
Epoch 5/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 565s 797ms/step - accuracy: 0.7128 - loss: 0.7195 - val_accuracy: 0.7624 - val_loss: 0.6135
Epoch 6/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 652s 914ms/step - accuracy: 0.7215 - loss: 0.6972 - val_accuracy: 0.7519 - val_loss: 0.6149
Epoch 7/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 535s 755ms/step - accuracy: 0.7201 - loss: 0.6922 - val_accuracy: 0.7374 - val_loss: 0.6605
Epoch 8/10
705/705 ━━━━━━━━━━━━━━━━━━━━ 722s 1s/step - accuracy: 0.7291 - loss: 0.6679 

In [ ]:
# Plot comparison of accuracy for each model for each crop
for crop, crop_results in results.items():
    plt.figure(figsize=(12, 6))
    for model_name, result in crop_results.items():
        plt.plot(result['history'].history['val_accuracy'] + result['history_fine'].history['val_accuracy'], label=model_name)

    plt.title(f'Model validation accuracy comparison for {crop}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.show()

    # Print test accuracy for each model
    for model_name, result in crop_results.items():
        print(f'{crop} - {model_name} Test Accuracy: {result["accuracy"]*100:.2f}%')

In [ ]:
# Display some correctly and incorrectly classified images
def display_classification_results(model, generator, num_images=5):
    class_labels = list(generator.class_indices.keys())
    images, labels = next(generator)
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels, axis=1)
    
    fig, axes = plt.subplots(2, num_images, figsize=(20, 8))
    fig.suptitle('Correctly and Incorrectly Classified Images', fontsize=16)
    
    correct = np.where(predicted_classes == true_classes)[0]
    incorrect = np.where(predicted_classes != true_classes)[0]
    
    for i in range(num_images):
        if i < len(correct):
            axes[0, i].imshow(images[correct[i]])
            axes[0, i].set_title(f'True: {class_labels[true_classes[correct[i]]]}, Pred: {class_labels[predicted_classes[correct[i]]]}')
            axes[0, i].axis('off')
        if i < len(incorrect):
            axes[1, i].imshow(images[incorrect[i]])
            axes[1, i].set_title(f'True: {class_labels[true_classes[incorrect[i]]]}, Pred: {class_labels[predicted_classes[incorrect[i]]]}')
            axes[1, i].axis('off')

    plt.show()

In [ ]:
# Display results for MobileNetV2 for each crop
for crop, crop_results in results.items():
    print(f'Displaying results for {crop} - MobileNetV2')
    display_classification_results(crop_results['MobileNetV2']['model'], test_generator)